In [2]:
import ee

# Authenticate using your Google account
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()




Successfully saved authorization token.


In [3]:
guincho_area = ee.Geometry.Rectangle([-9.5, 38.7, -9.4, 38.8])


In [6]:
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(guincho_area) \
    .filterDate('2020-01-01', '2020-12-31') \
    .sort('CLOUD_COVER')
# Filter the Sentinel-2 collection to exclude images with missing or invalid WRS_PATH
sentinel2_collection = sentinel2_collection.filterMetadata('WRS_PATH', 'not_equals', None)

# Create a median composite of the Sentinel-2 images
sentinel2_composite = sentinel2_collection.median()

 

In [7]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a map object centered on the Guincho area
guincho_map = folium.Map(location=[38.733, -9.47], zoom_start=12)

# Add the Sentinel-2 true-color image layer
true_color_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
guincho_map.add_ee_layer(sentinel2_composite, true_color_vis, 'True Color')

# Add a layer control and display the map
guincho_map.add_child(folium.LayerControl())
guincho_map


In [16]:
# Define the center point of Guincho and the buffer distance in meters
center_point = ee.Geometry.Point([-9.475831, 38.729229])
buffer_distance = 500

# Define the width and height of the rectangle (1km x 2km)
width = 1000
height = 2000

# Create a rectangle around the center point
rectangle = center_point.buffer(buffer_distance).bounds()

# Calculate the number of points in the grid (resolution: 100 meters)
grid_resolution = 100
num_points_x = int(width / grid_resolution)
num_points_y = int(height / grid_resolution)

# Create a grid of points within the rectangle
points = ee.FeatureCollection.randomPoints(rectangle, num_points_x * num_points_y)


 

####

# Filtering Sentinel-2 collection
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(rectangle) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.1)

# Select the bands and compute the median
sentinel2_median = sentinel2_collection.select(['B2', 'B3', 'B4', 'B8']).median()

# Sample the data at the points
terrain_data = sentinel2_median.sampleRegions(collection=points, geometries=True, scale=10) # scale set to 10 meters to match Sentinel-2 resolution

# Retrieve the results as a list of dictionaries
results = terrain_data.getInfo()['features']


In [19]:
def classify_land_cover(results):
    land_cover = []

    for feature in results:
        coordinates = feature['geometry']['coordinates']
        properties = feature['properties']
        blue = properties['B2']
        green = properties['B3']
        red = properties['B4']
        nir = properties['B8']  # Assuming you included the NIR band in the image collection and sampling process

        # Calculate indices
        ndvi = (nir - red) / (nir + red)
        ndwi = (green - nir) / (green + nir)
        bui = (nir + red) - (blue + green)
        bsi = ((red - blue) / (red + blue)) - ((nir - green) / (nir + green))

        # Classify land cover based on indices
        if ndvi > 0.3:
            cover_type = "Vegetation"
        elif ndwi > 0.3:
            cover_type = "Water"
        elif bui > 0:
            cover_type = "Roads/Buildings"
        elif bsi > 0:
            cover_type = "Bare Soil/Rocks"
        else:
            cover_type = "Unknown"

        land_cover.append((coordinates, cover_type))

    return land_cover


classified_land_cover = classify_land_cover(results)
for coords, cover_type in classified_land_cover:
    print(f"Coordinates: {coords[::-1]}, Land Cover: {cover_type}")


Coordinates: [38.728213444725014, -9.47188127152044], Land Cover: Roads/Buildings
Coordinates: [38.72497950970219, -9.470893124707908], Land Cover: Roads/Buildings
Coordinates: [38.726596477213604, -9.476732174054685], Land Cover: Roads/Buildings
Coordinates: [38.72560833040107, -9.470893124707908], Land Cover: Roads/Buildings
Coordinates: [38.73297451573085, -9.476193184884215], Land Cover: Water
Coordinates: [38.733693167958144, -9.472060934577264], Land Cover: Roads/Buildings
Coordinates: [38.73081855904896, -9.476283016412626], Land Cover: Water
Coordinates: [38.73270502114561, -9.48014577213434], Land Cover: Unknown
Coordinates: [38.73153721127626, -9.480235603662752], Land Cover: Water
Coordinates: [38.73045923293532, -9.476822005583099], Land Cover: Water
Coordinates: [38.72839310778184, -9.472150766105676], Land Cover: Roads/Buildings
Coordinates: [38.73099822210579, -9.473767733617091], Land Cover: Roads/Buildings
Coordinates: [38.729201591537546, -9.470174472480613], Land Cov